In [2]:
import boto3
import botocore
import sagemaker
import sys
import time

!pip install sagemaker --upgrade -q

In [12]:
session = sagemaker.Session()
runtime = boto3.client("sagemaker-runtime")
client = boto3.client("sagemaker")
region = session.boto_region_name
prefix='spectra-test'
role = "AmazonSageMaker-ExecutionRole-20240618T160945"
account_id = boto3.client('sts').get_caller_identity().get('Account')

In [10]:
# Set an unique endpoint config name
endpoint_config_name = f"{prefix}-endpoint-config"
print(f"Demo endpoint config name: {endpoint_config_name}")

# Set varient name and instance type for hosting
variant_name = "AllTraffic"
instance_type = "ml.g5.2xlarge"
model_data_download_timeout_in_seconds = 3600
container_startup_health_check_timeout_in_seconds = 3600

initial_instance_count = 1
max_instance_count = 2
print(f"Initial instance count: {initial_instance_count}")
print(f"Max instance count: {max_instance_count}")

client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ExecutionRoleArn=f"arn:aws:iam::{account_id}:role/service-role/{role}",
    ProductionVariants=[
        {
            "VariantName": variant_name,
            "InstanceType": instance_type,
            "InitialInstanceCount": 1,
            "ModelDataDownloadTimeoutInSeconds": model_data_download_timeout_in_seconds,
            "ContainerStartupHealthCheckTimeoutInSeconds": container_startup_health_check_timeout_in_seconds,
            "ManagedInstanceScaling": {
                "Status": "ENABLED",
                "MinInstanceCount": initial_instance_count,
                "MaxInstanceCount": max_instance_count,
            },
            "RoutingConfig": {"RoutingStrategy": "LEAST_OUTSTANDING_REQUESTS"},
        }
    ],
)

Demo endpoint config name: spectra-test-endpoint-config
Initial instance count: 1
Max instance count: 2


{'EndpointConfigArn': 'arn:aws:sagemaker:us-east-1:452706865406:endpoint-config/spectra-test-endpoint-config',
 'ResponseMetadata': {'RequestId': 'bcf71581-122d-4486-97d0-db2d5f9dcc53',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'bcf71581-122d-4486-97d0-db2d5f9dcc53',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '109',
   'date': 'Mon, 01 Jul 2024 10:02:29 GMT'},
  'RetryAttempts': 0}}

In [14]:
endpoint_name = f"{prefix}-endpoint"
print(f"Demo endpoint name: {endpoint_name}")

client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name,
)

Demo endpoint name: spectra-test-endpoint


{'EndpointArn': 'arn:aws:sagemaker:us-east-1:452706865406:endpoint/spectra-test-endpoint',
 'ResponseMetadata': {'RequestId': 'e5638ed6-9b7a-47e3-a241-358821838b47',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'e5638ed6-9b7a-47e3-a241-358821838b47',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '89',
   'date': 'Mon, 01 Jul 2024 10:04:09 GMT'},
  'RetryAttempts': 0}}

In [15]:
session.wait_for_endpoint(endpoint_name)

--!

{'EndpointName': 'spectra-test-endpoint',
 'EndpointArn': 'arn:aws:sagemaker:us-east-1:452706865406:endpoint/spectra-test-endpoint',
 'EndpointConfigName': 'spectra-test-endpoint-config',
 'ProductionVariants': [{'VariantName': 'AllTraffic',
   'CurrentInstanceCount': 1,
   'DesiredInstanceCount': 1,
   'ManagedInstanceScaling': {'Status': 'ENABLED',
    'MinInstanceCount': 1,
    'MaxInstanceCount': 2},
   'RoutingConfig': {'RoutingStrategy': 'LEAST_OUTSTANDING_REQUESTS'}}],
 'EndpointStatus': 'InService',
 'CreationTime': datetime.datetime(2024, 7, 1, 6, 4, 9, 564000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2024, 7, 1, 6, 6, 49, 282000, tzinfo=tzlocal()),
 'ResponseMetadata': {'RequestId': 'd32a0bed-2bd0-420d-a88a-9640b653a908',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'd32a0bed-2bd0-420d-a88a-9640b653a908',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '532',
   'date': 'Mon, 01 Jul 2024 10:07:14 GMT'},
  'RetryAttempts